<a href="https://colab.research.google.com/github/margual56/InteligenciaNegocioClasificadores/blob/main/Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imprime el tiempo que tarda en ejecutar cada celda
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 40.0 MB/s 
time: 457 µs (started: 2022-11-17 10:36:33 +00:00)


In [2]:
# Instala el clasificador y open
!pip install tabpfn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 31.6 MB/s 
     |████████████████████████████████| 361 kB 46.5 MB/s 
time: 4.74 s (started: 2022-11-17 10:36:33 +00:00)


In [3]:
import sys
import numpy as np
from pathlib import Path
import pandas as pd
import torch
import os
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pathlib import Path

from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay

# Limit the number of rows
limit = 100

time: 3.57 s (started: 2022-11-17 10:36:38 +00:00)


In [5]:
# Cargar los datos (https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))
X, y = fetch_covtype(return_X_y=True)

X = pd.DataFrame(X).sample(n=limit, random_state = 2)
y = pd.DataFrame(y).sample(n=limit, random_state = 2)

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

(100, 54)
(100, 1)
time: 768 ms (started: 2022-11-17 10:38:45 +00:00)


In [6]:
# N_ensemble --> The more, the better and slower
tab = TabPFNClassifier(device='cuda', N_ensemble_configurations=16)

We have to download the TabPFN, as there is no checkpoint at  /usr/local/lib/python3.7/dist-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
It has about 100MB, so this might take a moment.
Using style prior: True
Using cuda device
Using a Transformer with 25.82 M parameters
time: 5.13 s (started: 2022-11-17 10:38:51 +00:00)


In [12]:
tab.fit(X_train, y_train.values.ravel(), overwrite_warning=True)

TabPFNClassifier(N_ensemble_configurations=16, device='cuda')

time: 8.77 ms (started: 2022-11-17 10:41:43 +00:00)


In [8]:
y_evaltab, p_evaltab = tab.predict(X_test, return_winning_probability=True)
print('TabPFN ccuracy', accuracy_score(y_test, y_evaltab))

TabPFN ccuracy 0.48484848484848486
time: 2.98 s (started: 2022-11-17 10:39:02 +00:00)


In [9]:
rfr = RandomForestClassifier()

time: 735 µs (started: 2022-11-17 10:39:09 +00:00)


In [13]:
rfr.fit(X_train, y_train.values.ravel())

RandomForestClassifier()

time: 186 ms (started: 2022-11-17 10:41:47 +00:00)


In [11]:
y_evalrfr = rfr.predict(X_test)
print('RandForest (100% dataset) Accuracy', accuracy_score(y_test, y_evalrfr))

RandForest (100% dataset) Accuracy 0.6363636363636364
time: 20.9 ms (started: 2022-11-17 10:39:14 +00:00)
